## Preprocessing orbital queries

### Goals

 `orbital` is able to translate an entire scikitlearn pipeline to SQL, including feature transformation and prediction. However, it may be preferable to have these steps separate for multiple reasons: 
 
 - `orbital` doesn't do feature transformations in a subquery, so if the same variable is used many times the transformation is inefficient
 - it is hard to do assertions and real-time tests in SQL versus python, so creating features separately 
 - broadly we can just have more compact and readable SQL

 This notebook shows how to conduct these processes in separate steps and demonstrates the equivalency of the results. 


### Set Up

In [57]:
import orbital
import duckdb
import sqlglot
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

In [58]:
#| label: model-prep

# make data dataset
X_train, y_train = make_classification(10000, random_state = 102)
X_train = X_train.round(3)

# get column names for use in pipeline
n_cols = len(X_train[0])
nm_cols = [f"f{i}" for i in range(n_cols)]
feat_dict = {c:orbital.types.DoubleColumnType() for c in nm_cols}

# fit sklearn pipeline
pipeline = Pipeline([
  ("scaler", StandardScaler()),
  ("clf", RandomForestClassifier(max_depth = 1, n_estimators = 1, random_state = 504)),
])
pipeline.fit(X_train, y_train)

,steps,"[('scaler', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,n_estimators,1
,criterion,'gini'
,max_depth,1
,min_samples_split,2


### Run `orbital`

We can see orbital does the feature transformations in-line. This may result in applying logic to the features multiple times and limits our ability to check them for correctness. 

In [59]:
orbital_pipeline = orbital.parse_pipeline(pipeline, features=feat_dict)
sql_raw = orbital.export_sql("DATA_TABLE", orbital_pipeline, dialect="duckdb")

c:\Users\emily\Desktop\orbital-exploration\.venv\Lib\site-packages\orbital\translation\steps\trees\classifier.py:135: FutureWarning: `case` is deprecated as of v10.0.0, removed in v11.0; use ibis.cases()
  ibis.case().when(condition, t_val).else_(f_val).end()
c:\Users\emily\Desktop\orbital-exploration\.venv\Lib\site-packages\orbital\translation\steps\trees\classifier.py:157: FutureWarning: `case` is deprecated as of v10.0.0, removed in v11.0; use ibis.cases()
  ibis.case()


In [60]:
print(
    sqlglot.transpile(sql_raw, write="duckdb", identify=True, pretty=True)[0]
)

SELECT
  CAST(CASE
    WHEN CASE
      WHEN (
        (
          "t0"."f15" - -0.001669299999999982
        ) / 1.00359349127897
      ) <= -2.0449821949005127
      THEN 0.3595505654811859
      ELSE 0.4991855025291443
    END > 0.5
    THEN 1
    ELSE 0
  END AS BIGINT) AS "output_label",
  1.0 - CASE
    WHEN (
      (
        "t0"."f15" - -0.001669299999999982
      ) / 1.00359349127897
    ) <= -2.0449821949005127
    THEN 0.3595505654811859
    ELSE 0.4991855025291443
  END AS "output_probability.0",
  CASE
    WHEN (
      (
        "t0"."f15" - -0.001669299999999982
      ) / 1.00359349127897
    ) <= -2.0449821949005127
    THEN 0.3595505654811859
    ELSE 0.4991855025291443
  END AS "output_probability.1"
FROM "DATA_TABLE" AS "t0"
SELECT
  CAST(CASE
    WHEN CASE
      WHEN (
        (
          "t0"."f15" - -0.001669299999999982
        ) / 1.00359349127897
      ) <= -2.0449821949005127
      THEN 0.3595505654811859
      ELSE 0.4991855025291443
    END > 0.5
    THEN 1
  

### Render Separate SQL for Feature Transformation & Model Scoring

In [61]:
# fit a two part pipeline
ppl_pre = Pipeline([ ("scaler", StandardScaler()) ])
ppl_clf =   pipeline = Pipeline([
  ("preprocess", ColumnTransformer([("scaler", StandardScaler(), [])], remainder="passthrough")),
  ("clf", RandomForestClassifier(max_depth = 1, n_estimators = 1, random_state = 504)),
    ])
ppl_pre.fit(X_train)
X_trans = ppl_pre.transform(X_train)
ppl_clf.fit(X_trans, y_train)

,steps,"[('preprocess', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('scaler', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [62]:
preds_ppl = pipeline.predict_proba(X_train)[:,1]
preds_clf = pipeline.predict_proba(X_trans)[:,1]

print(f"ppl and clf match: {np.all(np.isclose(preds_ppl, preds_clf))}")
print(f"ppl and clf prop agree: {np.sum(np.isclose(preds_ppl, preds_clf)) / len(preds_ppl):.3f}")

ppl and clf match: False
ppl and clf prop agree: 0.999
ppl and clf match: False
ppl and clf prop agree: 0.999


In [63]:
orb_pre = orbital.parse_pipeline(ppl_pre, features=feat_dict)
sql_pre = orbital.export_sql("DATA_TABLE", orb_pre, dialect="duckdb")
print(sql_pre)
sql_pre = sql_pre.replace("variable.", "variable_")
print(sql_pre)

SELECT ("t0"."f0" - -0.008924600000000005) / 0.9991851393584894 AS "variable.f0", ("t0"."f1" - -0.012443899999999983) / 1.0037471353148564 AS "variable.f1", ("t0"."f2" - 0.0023704999999999894) / 0.9981629063583519 AS "variable.f2", ("t0"."f3" - -0.006932800000000027) / 1.0031342737062483 AS "variable.f3", ("t0"."f4" - 0.017726800000000018) / 1.474398087275536 AS "variable.f4", ("t0"."f5" - -0.005110599999999954) / 1.0030191017959942 AS "variable.f5", ("t0"."f6" - -0.006099800000000014) / 1.011045355678943 AS "variable.f6", ("t0"."f7" - -0.00709279999999996) / 0.9945345624904943 AS "variable.f7", ("t0"."f8" - -0.02168959999999997) / 0.9992444634081482 AS "variable.f8", ("t0"."f9" - 0.01237740000000001) / 0.9846936678831847 AS "variable.f9", ("t0"."f10" - 0.0017515000000000104) / 1.415451619359612 AS "variable.f10", ("t0"."f11" - 0.00010549999999999735) / 0.9875974915772884 AS "variable.f11", ("t0"."f12" - -0.000915299999999986) / 0.9909802931067332 AS "variable.f12", ("t0"."f13" - -0.03

In [64]:
feat_dict_postproc = {f"variable_{k}":v for k,v in feat_dict.items()}
orb_clf = orbital.parse_pipeline(ppl_clf, features=feat_dict_postproc)
sql_clf = orbital.export_sql("FEATURES", orb_clf, dialect="duckdb")
print(sql_clf)

SELECT CAST(CASE WHEN CASE WHEN "t0"."variable_f15" <= -2.0449821949005127 THEN 0.3595505654811859 ELSE 0.4991855025291443 END > 0.5 THEN 1 ELSE 0 END AS BIGINT) AS "output_label", 1.0 - CASE WHEN "t0"."variable_f15" <= -2.0449821949005127 THEN 0.3595505654811859 ELSE 0.4991855025291443 END AS "output_probability.0", CASE WHEN "t0"."variable_f15" <= -2.0449821949005127 THEN 0.3595505654811859 ELSE 0.4991855025291443 END AS "output_probability.1" FROM "FEATURES" AS "t0"
SELECT CAST(CASE WHEN CASE WHEN "t0"."variable_f15" <= -2.0449821949005127 THEN 0.3595505654811859 ELSE 0.4991855025291443 END > 0.5 THEN 1 ELSE 0 END AS BIGINT) AS "output_label", 1.0 - CASE WHEN "t0"."variable_f15" <= -2.0449821949005127 THEN 0.3595505654811859 ELSE 0.4991855025291443 END AS "output_probability.0", CASE WHEN "t0"."variable_f15" <= -2.0449821949005127 THEN 0.3595505654811859 ELSE 0.4991855025291443 END AS "output_probability.1" FROM "FEATURES" AS "t0"


c:\Users\emily\Desktop\orbital-exploration\.venv\Lib\site-packages\orbital\translation\steps\trees\classifier.py:135: FutureWarning: `case` is deprecated as of v10.0.0, removed in v11.0; use ibis.cases()
  ibis.case().when(condition, t_val).else_(f_val).end()
c:\Users\emily\Desktop\orbital-exploration\.venv\Lib\site-packages\orbital\translation\steps\trees\classifier.py:157: FutureWarning: `case` is deprecated as of v10.0.0, removed in v11.0; use ibis.cases()
  ibis.case()


### Testing Output

We can double check that the reformatted query remains valid for prediction.

In [65]:
DATA_TABLE = pd.DataFrame(X_train, columns = nm_cols)
con = duckdb.connect(':memory:')
con.execute('')
con.execute(f'create or replace view ONE_STEP as {sql_raw}')
con.execute(f'create or replace view FEATURES as {sql_pre}')
con.execute(f'create or replace view PREDICTION as {sql_clf}')

In [66]:
df_onestep = con.sql('SELECT "output_probability.1" as pred FROM ONE_STEP').df()
df_twostep = con.sql('SELECT "output_probability.1" as pred FROM PREDICTION').df()

In [67]:
preds_1 = df_onestep['pred']
preds_2 = df_twostep['pred']

print(f"1-step and 2-step match: {np.all(np.isclose(preds_1, preds_2))}")
print(f"ppl and clf prop agree: {np.sum(np.isclose(preds_1, preds_2)) / len(preds_1):.3f}")

1-step and 2-step match: True
ppl and clf prop agree: 1.000
1-step and 2-step match: True
ppl and clf prop agree: 1.000
